In [1]:
import os
import sys
# sys.path.append("../virtuosoNet")
# sys.path.append("C:\\Users\\fosca\\Desktop\\CNAM\\midi_beat_tracking\\FMP_0.1.0")

from pathlib import Path
import pandas as pd
import music21 as m21
import pretty_midi as pm
from  matplotlib import pyplot as plt
# import LibFMP.B
# import LibFMP.C2
# import LibFMP.C6
%matplotlib inline
import numpy as np
import math
import pandas as pd
import librosa
import soundfile

import IPython.display as ipd

In [2]:
def cleaned_m2m_df(midi2midi_path):
    m2mdf = pd.read_csv(midi2midi_path,sep="\t",skiprows=1,header=None, index_col=False)
    m2mdf.columns =["alignID", "alignOntime", "alignSitch" ,"alignPitch", "alignOnvel", "refID", "refOntime", "refSitch", "refPitch", "refOnvel","empty"]
    #drop the last column "empty"
    m2mdf = m2mdf.drop(columns=["empty"])
    #drop the rows where the alignOntime is -1 (note not existing in both files midi)
    return m2mdf[m2mdf["alignOntime"] != -1]

def generate_performance_annotations(m2m_df, cl_quant_ann):
    #create the list with the time of all annotations
    quant_ann_time_list = cl_quant_ann["time"].tolist()
    quant_ann_type_list = cl_quant_ann["type"].tolist()
    perf_ann_list = []
    for i,at in enumerate(quant_ann_time_list):
        p_ons=list(m2m_df[(m2m_df["refOntime"]>at-0.020) & (m2m_df["refOntime"]<at+0.020)]["alignOntime"]) #40 ms accepted window
        if len(p_ons) == 0: #no note matching the ann position in the performance
            perf_ann_list.append(None)
            print("missing corrispondency at quant time", at)
        else:
            perf_ann_list.append(np.mean(p_ons))

    # clean the values that are smaller than the value before
    for ind,at in enumerate(perf_ann_list[1:]):
        if (ind >0) and (at is not None) and (perf_ann_list[ind-1] is not None) and (at < perf_ann_list[ind-1]) :
            print("Unexpected value",db,"smaller than the previous",perf_ann_list[ind-1])
            perf_ann_list[ind] = None
            
    #interpolate to fill the None values
    filled_at = pd.Series(unquant_dbs).interpolate().tolist()
    
    #associate the type of annotation
    assert(len(quant_ann_type_list) == len(filled_at))
    perd_ann_dict = dict(zip(filled_at, quant_ann_type_list))
    
    return perd_ann_dict

In [ ]:
df = pd.read_